# 🎙️ AutomationX TTS API

⚠️ **Runtime > Change runtime type > T4 GPU**

**Adım 1:** Kurulum → **Adım 2:** Başlat

## 1️⃣ Kurulum

In [ ]:
# Temizlik
!pip uninstall -y torch torchvision torchaudio transformers chatterbox-tts accelerate huggingface-hub diffusers torchao perth resemble-perth 2>/dev/null

# PyTorch + CUDA
!pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121

# Bağımlılıklar
!pip install -q transformers==4.46.3 diffusers==0.29.0 'huggingface_hub>=0.23.0' 'accelerate>=0.25.0'
!pip install -q 'numpy>=1.24.0,<1.26.0' librosa safetensors soundfile scipy
!pip install -q resemble-perth s3tokenizer conformer
!pip install -q fastapi uvicorn python-multipart python-dotenv
!pip install -q chatterbox-tts --no-deps
!pip install -q protobuf==3.20.3

# Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb 2>/dev/null

# Repo
!git clone https://github.com/OsmanYavuz-web/AutomationX-TTS.git /content/AutomationX-TTS 2>/dev/null || (cd /content/AutomationX-TTS && git pull)

print("\n✅ Kurulum tamamlandı! Kernel restart...")
get_ipython().kernel.do_shutdown(True)

## 2️⃣ Başlat

In [ ]:
import sys, os, subprocess, threading, time, re, uvicorn

REPO = "/content/AutomationX-TTS"
PORT = 7860

sys.path.insert(0, REPO)
os.chdir(REPO)

# .env
with open(".env", "w") as f:
    f.write("PORT=7860\nIDLE_TIMEOUT=0\nLOG_TYPE=console")
os.makedirs("outputs", exist_ok=True)

# Model
print("🔄 Model yükleniyor...")
from api import api_app
from core import get_state
_ = get_state().tts_model
print("✅ Model yüklendi!")

# Server
threading.Thread(target=lambda: uvicorn.run(api_app, host="0.0.0.0", port=PORT, log_level="warning"), daemon=True).start()
time.sleep(2)

# Tunnel
p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"], stderr=subprocess.PIPE, text=True)
for _ in range(30):
    line = p.stderr.readline()
    if "trycloudflare.com" in line:
        url = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
        if url:
            print(f"\n{'='*50}")
            print(f"🌐 API:  {url.group(0)}")
            print(f"📚 Docs: {url.group(0)}/docs")
            print(f"{'='*50}")
            break
    time.sleep(0.3)

print("\n⏳ API çalışıyor. Durdurmak: Runtime > Interrupt")